In [7]:
from sklearn.datasets import load_iris
import random
import numpy as np

iris = load_iris()
X, y = iris.data, iris.target

iris_data = list(zip(X, y))

split_ratio = 0.8

num_samples = len(iris_data)
num_train_samples = int(num_samples * split_ratio)
num_test_samples = num_samples - num_train_samples

random.shuffle(iris_data)

train_data = iris_data[:num_train_samples]
test_data = iris_data[num_train_samples:]

X_train = [data[0] for data in train_data]
y_train = [data[1] for data in train_data]

X_test = [data[0] for data in test_data]
y_test = [data[1] for data in test_data]

X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)


In [11]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def compute_cost(y_true, y_pred):
    m = len(y_true)
    cost = (-1/m) * np.sum(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
    return cost

def logistic_regression(X, y, learning_rate=0.01, num_iterations=10000):
    m, n = X.shape
    weights = np.zeros(n)
    bias = 0
    
    for _ in range(num_iterations):
        linear_model = np.dot(X, weights) + bias
        y_pred = sigmoid(linear_model)
   
        dw = (1/m) * np.dot(X.T, (y_pred - y))
        db = (1/m) * np.sum(y_pred - y)
        
        weights -= learning_rate * dw
        bias -= learning_rate * db
        
    return weights, bias


In [12]:
from sklearn.model_selection import KFold

def predict(X, weights, bias):
    y_pred = sigmoid(np.dot(X, weights) + bias)
    return np.round(y_pred)

kf = KFold(n_splits=5, shuffle=True)

best_fold = -1
best_accuracy = 0

for fold, (train_index, test_index) in enumerate(kf.split(X_train)):
    X_train_fold, X_valid_fold = X_train[train_index], X_train[test_index]
    y_train_fold, y_valid_fold = y_train[train_index], y_train[test_index]

    split_idx = int(0.9 * len(X_train_fold))
    X_subtrain, X_subvalid = X_train_fold[:split_idx], X_train_fold[split_idx:]
    y_subtrain, y_subvalid = y_train_fold[:split_idx], y_train_fold[split_idx:]

    weights, bias = logistic_regression(X_subtrain, y_subtrain)
    
    y_valid_pred = predict(X_subvalid, weights, bias)
    
    accuracy = compute_accuracy(y_subvalid, y_valid_pred)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_fold = fold
        
    print(f"Fold {fold + 1}: Accuracy = {accuracy:.2f}")

print(f"Best Fold: {best_fold + 1} with Accuracy = {best_accuracy:.2f}")


Fold 1: Accuracy = 0.30
Fold 2: Accuracy = 0.10
Fold 3: Accuracy = 0.30
Fold 4: Accuracy = 0.20
Fold 5: Accuracy = 0.30
Best Fold: 1 with Accuracy = 0.30
